In [44]:
import csv


In [66]:
def extract_metadata():
    path = ""
    metafile = open('files/metadata.txt', 'r') 
    metalines = metafile.readlines() 
    tables_dict = {}
    is_rec = False
    is_tname = False
    cur_table = ""
    for line in metalines:
        if line.startswith('<begin_table>'):
            is_rec = True
            is_tname = True
        elif line.startswith('<end_table>'):
            if_rec = False
        elif is_tname:
            cur_table = str(line).strip()
            tables_dict[cur_table] = {}
            is_tname = False
        else:
            tables_dict[cur_table][str(line).strip()] = []
    return tables_dict
            
def extract_csvdata(tables_dict):
    tables_data = tables_dict
    for tn,cl in tables_dict.items(): 
        with open('files/'+tn+'.csv', newline='') as table_file:
            all_data = csv.reader(table_file,delimiter=',')
#             print(all_data)
            for row in all_data:
                for c,i in zip(cl,range(len(cl))):
                    tables_data[tn][c].append(int(row[i]))
    return tables_data
    

In [67]:
tables_dict = extract_metadata()
print(tables_dict)
tables_data = extract_csvdata(tables_dict)
print(tables_data)

{'table1': {'A': [], 'B': [], 'C': []}, 'table2': {'D': [], 'E': []}}
{'table1': {'A': [922, 640, 775, -551, -952, -354, -497, 411, -900, 858], 'B': [158, 773, 85, 811, 311, 646, 335, 803, 718, 731], 'C': [5727, 5058, 10164, 1534, 1318, 7063, 4549, 10519, 9020, 3668]}, 'table2': {'D': [158, 773, 85, 811, 311, 646, 335, 803, 718, 731], 'E': [11191, 14421, 5117, 13393, 16116, 5403, 6309, 12262, 10226, 13021]}}
